# Culture Measures Based on Company Reviews

In [2]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import nltk, re, math, csv
# nltk.download('wordnet')
# nlkt.download('punkt')

import koolture as kt

from string import punctuation
from functools import partial
import concurrent.futures as cf
from collections import defaultdict

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

%load_ext autoreload
%autoreload 2

Load your dataset.

In [3]:
df = pd.read_csv('../data/clean_gs.csv')
df.head()

,employer,id,pros,cons
0,American Express,44001,Still not big enough in market place,"Great brand , Good leadership , Clear business..."
1,Eventum IT Solutions,44004,Nothing important on my point of view.,"Learn new technologies, helpful people, good m..."
2,Eventum IT Solutions,44004,Alot of friends working together which isn't v...,Very good opportunities to learn technologies
3,Eventum IT Solutions,44004,Working hours are not good and need to add the...,You can learn technically a lot in this company.
4,Eventum IT Solutions,44004,No Real Cons at all,- Very friendly environment.\r\n- Highly exper...


In [4]:
df.shape

(78420, 4)

## Fist Range of Topics

In [5]:
our_range = 2, 10, 50, 100, 150, 200, 250, 300

Remove the company names from the reviews, and extract the reviews into a numpy array.

In [6]:
comps_of_interest = df.employer.value_counts()
comps_of_interest.head(8)

Amazon               561
Oracle               422
Microsoft            349
Siemens              343
Dell Technologies    338
IBM                  337
EY                   324
PwC                  300
Name: employer, dtype: int64

In [7]:
#comps_of_interest = (comps_of_interest).index
comps_of_interest = (comps_of_interest[(comps_of_interest == 48)]).index
len(comps_of_interest), comps_of_interest

(5,
 Index(['CGI', 'Alstom Transportation', 'GlobalLogic', 'Goldman Sachs', '3M'], dtype='object'))

In [8]:
cond2 = df['employer'].isin(comps_of_interest) # create the condition
df_interest = df[cond2].copy() # get the new dataset
unique_ids = df_interest['employer'].unique() # get the unique IDs or unique employers in the dataset
unique_ids

array(['GlobalLogic', 'Alstom Transportation', 'Goldman Sachs', '3M',
       'CGI'], dtype=object)

In [9]:
df_interest.shape

(240, 4)

In [10]:
reviews_nums = df_interest['employer'].value_counts().reset_index()
reviews_nums.columns = ['employerID', 'reviews_nums']
reviews_nums.head()

,employerID,reviews_nums
0,GlobalLogic,48
1,Goldman Sachs,48
2,3M,48
3,Alstom Transportation,48
4,CGI,48


## Fix Custom Stopwords List Before Cleaning

The text preprocessing of the corpus takes place in parallel. You first normalize the reviews and then take the root of the words.

In [11]:
data_pros = df_interest['pros'].values
stopwords = nltk.corpus.stopwords.words('english') + [token.lower() for token in unique_ids]
stopwords[-10:]

["weren't",
 'won',
 "won't",
 'wouldn',
 "wouldn't",
 'globallogic',
 'alstom transportation',
 'goldman sachs',
 '3m',
 'cgi']

In [12]:
normalize_doc = partial(kt.normalize_doc, stopwords=stopwords)

In [13]:
%%time

with cf.ProcessPoolExecutor() as e:
    data_pros_cleaned = e.map(normalize_doc, data_pros)
    data_pros_cleaned = list(e.map(kt.root_of_word, data_pros_cleaned))

df_interest['pros_clean'] = data_pros_cleaned

CPU times: user 123 ms, sys: 75.3 ms, total: 198 ms
Wall time: 265 ms


## Create Vectorizers Tuple

In [14]:
%%time

vectorizers_dicts = kt.get_vectorizers(data=df_interest, unique_ids=unique_ids,
                                      company_col='employer', reviews_col='pros', 
                                      vrizer=CountVectorizer())

CPU times: user 21 ms, sys: 4.86 ms, total: 25.8 ms
Wall time: 33.6 ms


The following block run the models in parallel over the companies available and using the specifiedamount of topics in our_range variable and return a dictionary with the output of the get_models function for each company. It is used to identify the interval to search further for optimal topic number.

In [15]:
%%time

partial_func = partial(kt.get_models, topics=our_range, vrizer_dicts=vectorizers_dicts, unique_ids=unique_ids)

with cf.ProcessPoolExecutor() as e:
    output = list(e.map(partial_func, unique_ids))

CPU times: user 80.2 ms, sys: 125 ms, total: 205 ms
Wall time: 38.5 s


The next function will now iterate over the dictionary output from above, add each dataset into a list, and then concatenate them all into one dataset (output df contains exactly same information, but more readable, and used in next blocks).

In [16]:
output_df = kt.build_dataframe(output)
output_df.head()

,company,topics,coherence,models
0,GlobalLogic,2,0.123111,LatentDirichletAllocation(learning_method='onl...
1,GlobalLogic,10,0.197351,LatentDirichletAllocation(learning_method='onl...
2,GlobalLogic,50,0.199536,LatentDirichletAllocation(learning_method='onl...
3,GlobalLogic,100,0.179204,LatentDirichletAllocation(learning_method='onl...
4,GlobalLogic,150,0.127815,LatentDirichletAllocation(learning_method='onl...


The following loop iterates over the new dataframe, searches for the top 2 topics based on highest coherence, and appends to a list a tuple containing the company, a tuple with the top two topic numbers, and the fitted vectorizer from the original `vectorizers_list`.

In [17]:
%%time

topics_sorted, comps, tops = kt.top_two_topics(data=output_df, companies_var='company',
                               coherence_var='coherence', topics_var='topics',
                               unique_ids=unique_ids, vrizers_list=vectorizers_dicts.values())

CPU times: user 9.96 ms, sys: 2.62 ms, total: 12.6 ms
Wall time: 14.3 ms


Now run the `get_models` function again over the new space of topics. You will  need to
1. sort the tuple with the top two topics.
2. create a linearly spaced array with 10 elements between the top 2 topics, turn it into integers, make the array a set to eliminate any duplicates that might arise if there is a 2 in the top two topics, and then turn that into a list.
3. get your fixed partial function again
4. the output is the same as before

In [18]:
%%time


partial_func = partial(kt.get_models, 
                       vrizer_dicts=vectorizers_dicts, 
                       unique_ids=unique_ids)

with cf.ProcessPoolExecutor() as e:
    output2 = list(e.map(partial_func, comps, tops))

CPU times: user 50.8 ms, sys: 65.4 ms, total: 116 ms
Wall time: 10 s


Create multiple dataframes from dictionaries again and collapse them into 1.

In [19]:
output_df2 = kt.build_dataframe(output2)
output_df2.head()

,company,topics,coherence,models
0,GlobalLogic,32,0.224750,LatentDirichletAllocation(learning_method='onl...
1,GlobalLogic,36,0.225683,LatentDirichletAllocation(learning_method='onl...
2,GlobalLogic,41,0.233038,LatentDirichletAllocation(learning_method='onl...
3,GlobalLogic,10,0.197351,LatentDirichletAllocation(learning_method='onl...
4,GlobalLogic,45,0.217440,LatentDirichletAllocation(learning_method='onl...


Search for the best topic based on the new output, and get the top 10 words per topic. At the moment, you are only adding 1 of the topics for each company but you can change this by removing the indexing in `top_topics` below.

In [20]:
%%time

best_topics = kt.absolute_topics(output_df2, 'company', 'coherence', 
                                 'topics', 'models', vectorizers_dicts.values())

CPU times: user 5.02 ms, sys: 542 µs, total: 5.56 ms
Wall time: 6.08 ms


In [21]:
best_topics

defaultdict(tuple,
            {'GlobalLogic': (27,
              LatentDirichletAllocation(learning_method='online', max_iter=100,
                                        n_components=27, random_state=1234),
              0.25407249517297825),
             'Alstom Transportation': (41,
              LatentDirichletAllocation(learning_method='online', max_iter=100,
                                        n_components=41, random_state=1234),
              0.25718124236359025),
             'Goldman Sachs': (32,
              LatentDirichletAllocation(learning_method='online', max_iter=100,
                                        n_components=32, random_state=1234),
              0.2634840856910199),
             '3M': (36,
              LatentDirichletAllocation(learning_method='online', max_iter=100,
                                        n_components=36, random_state=1234),
              0.2656205072583462),
             'CGI': (18,
              LatentDirichletAllocation(learning_me

Check out your output.

Get the probabilities dataframes for each company and add them to a dictionary.

In [22]:
#generate matrix summarizing distribution of docs (reviews) over topics
docs_of_probas = defaultdict(pd.DataFrame)

for tup in vectorizers_dicts.values():
    docs_of_probas[tup[0]] = pd.DataFrame(best_topics[tup[0]][1].transform(tup[1]))

# Calculate the measures of interest

In [23]:
%%time

comP_h_results = defaultdict(float)
comT_h_results = defaultdict(float)
entropy_avg_results = defaultdict(float)
cross_entropy_results = defaultdict(float)

for company, proba_df in docs_of_probas.items():
    comP_h_results[company] = kt.comph(proba_df.values)
    comT_h_results[company] = kt.conth(proba_df)
    entropy_avg_results[company] = kt.ent_avg(proba_df.values)
    cross_entropy_results[company] = kt.avg_crossEnt(proba_df.values)

CPU times: user 1.55 s, sys: 13.4 ms, total: 1.56 s
Wall time: 1.57 s


In [24]:
comph_df = pd.DataFrame.from_dict(comP_h_results.items())
conth_df = pd.DataFrame.from_dict(comT_h_results.items())
crossEnt_df = pd.DataFrame.from_dict(cross_entropy_results.items())
cultureMetrics = comph_df.merge(conth_df, how = 'inner', right_on = 0, left_on = 0)
cultureMetrics = cultureMetrics.merge(crossEnt_df, how = 'inner', right_on = 0, left_on = 0)
cultureMetrics.columns = ['employerID', 'comph', 'conth', 'avgCrossEnt']
cultureMetrics.head()

,employerID,comph,conth,avgCrossEnt
0,GlobalLogic,0.589723,1.235161,8.182395
1,Alstom Transportation,0.613374,1.188420,8.972884
2,Goldman Sachs,0.605053,1.189686,8.646988
3,3M,0.612979,1.189208,8.886881
4,CGI,0.582782,1.193760,7.517453


In [25]:
df_best_topics = pd.DataFrame.from_records(best_topics).T.reset_index()
df_best_topics.columns = ['employerID', 'best_topic', 'model', 'coherence']
df_best_topics.head()

,employerID,best_topic,model,coherence
0,3M,36,LatentDirichletAllocation(learning_method='onl...,0.265621
1,Alstom Transportation,41,LatentDirichletAllocation(learning_method='onl...,0.257181
2,CGI,18,LatentDirichletAllocation(learning_method='onl...,0.278136
3,GlobalLogic,27,LatentDirichletAllocation(learning_method='onl...,0.254072
4,Goldman Sachs,32,LatentDirichletAllocation(learning_method='onl...,0.263484


In [26]:
(cultureMetrics.merge(reviews_nums, on='employerID', how='right')
               .merge(df_best_topics, on='employerID', how='right')
               .to_csv(f'culturemetrics_chunk_num.csv', index=False))

In [27]:
(cultureMetrics.merge(reviews_nums, on='employerID', how='right')
               )

,employerID,comph,conth,avgCrossEnt,reviews_nums
0,GlobalLogic,0.589723,1.235161,8.182395,48
1,Goldman Sachs,0.605053,1.189686,8.646988,48
2,3M,0.612979,1.189208,8.886881,48
3,Alstom Transportation,0.613374,1.188420,8.972884,48
4,CGI,0.582782,1.193760,7.517453,48


In [29]:
df_best_topics.merge(reviews_nums, on='employerID', how='right').head()

,employerID,best_topic,model,coherence,reviews_nums
0,GlobalLogic,27,LatentDirichletAllocation(learning_method='onl...,0.254072,48
1,Goldman Sachs,32,LatentDirichletAllocation(learning_method='onl...,0.263484,48
2,3M,36,LatentDirichletAllocation(learning_method='onl...,0.265621,48
3,Alstom Transportation,41,LatentDirichletAllocation(learning_method='onl...,0.257181,48
4,CGI,18,LatentDirichletAllocation(learning_method='onl...,0.278136,48


In [31]:
cultureMetrics.to_csv('CultureMetrics_TestSample_1000.csv', index=False)
(df_best_topics.merge(reviews_nums, on='employerID', how='right')
               .to_csv('best_topics.csv', index=False))